In [116]:
import os
import openai
import pandas as pd
import tiktoken
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

In [117]:
# Fine tuning increases the accuracy from 84% to 84.20%; minor change

In [118]:
df = pd.read_csv('../data/csv/processed_toxic_classification_dataset_short.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(int(x)))
df

,Text,Toxic
0,Geez forgetful Weve already discussed Marx ana...,0
1,Carioca RFA Thanks support request adminship f...,0
2,Birthday worries Enjoy ur daytalke,0
3,Pseudoscience category Im assuming article pse...,0
4,phrase exists would provided search engine eve...,0
...,...,...
995,2006 UTC rest us ever gone past 3RR good one V...,0
996,Yay lets Pedantic Semantics dance rolls eyes t...,0
997,supposed Know,0
998,guys really discuss napoleon need get fuckin l...,1


In [119]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [120]:
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

In [121]:
# initializing Pinecone vector database instance
docSearch = Pinecone(
    index_name=pc_index,
    embedding=OpenAIEmbeddings(openai_api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY)
)

In [122]:
temp_comment = "Classify these comments as either toxic (1) non-toxic (0). Toxic comments contain offensive, harmful, or abusive language. Label 1 for toxic and 0 for non-toxic \n\nExamples for Reference:"

In [123]:
# res = docSearch.max_marginal_relevance_search("please consider personal attack user Node eu Dear Node Eu take personal pride honor say commander well deserved Fuck yes know moment real identity please forgive wikipedia want assure family extend family mr Bortnikov harm intended untended occur long process well spend money involved realize finally deserve mercy Good look yes wikipedia last posting realize place muchmuch better real scientific world,0", k=25, fetch_k=10)
# text_one = res[1].page_content.split(' - ')
# text_two = res[2].page_content.split(' - ')
# text_three = res[3].page_content.split(' - ')
# text_four = res[4].page_content.split(' - ')
# text_five = res[5].page_content.split(' - ')

# temp_comment += '\n\n' + 'Text: \"' + text_one[0] + "\"" + '\n\n' + text_one[1] + '\n\n' +'Text: \"' + text_two[0] + "\"" + '\n\n' + text_two[1] + '\n\n'+'Text: \"' + text_three[0] + "\"" + '\n\n' + text_three[1] + '\n\n' + 'Text: \"' + text_four[0] + "\"" + '\n\n' + text_four[1] + '\n\n' + 'Text: \"' + text_five[0] + "\"" + '\n\n' + text_five[1] +'\n\n' + "Classify the following comment:"
# text_one
# # ['3868. Youre seriously moron 864433121', 'Label: 0']

In [124]:
with open('../data/text/temp.txt', 'w') as f:
    f.write(f"{temp_comment}\n")

In [125]:
def shot_additions(examples):
   evaluation_prompt = ''
   # Read the evaluation prompt from the text file with utf-8 encoding
   with open("../data/text/rag_prompts/gpt-3-5-1106-two-shot-prompt.txt", "r", encoding="utf-8") as file:
        evaluation_prompt += file.read() 
   evaluation_prompt += '\n\n'
   for example in examples:
       split_ex = example.split(' - ')
       comment = split_ex[0]
       label = split_ex[1]
       evaluation_prompt += 'Text: \"' + comment + "\"" + '\n\n' + label

   evaluation_prompt += '\n\nClassify the following comment:'

   return evaluation_prompt

In [126]:
def rag_prompt(text):
    two_shots = []
    search_results = docSearch.max_marginal_relevance_search(text, k=25, fetch_k=50)
    for i in search_results:
        content = i.page_content
        if i != 0 and len(content) < 50:
            two_shots.append(content)
        if len(two_shots) == 2:
            break
    return shot_additions(two_shots)

In [127]:
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [128]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../data/text/rag_prompts/gpt-3-5-1106-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [129]:
evaluation_prompt

'You are tasked with classifying comments as either toxic or non-toxic from a non-biased viewpoint. \nA comment is toxic if it contains harmful, abusive, or offensive language, such as threats, hate speech, \npersonal attacks, obscene language, or discrimination. ONLY comments with a 1 for toxic or a 0 for non-toxic. \nBelow are some examples and guidelines.\n\nExamples for Reference:\n\nText: “FUCK I"M SO EXCITEDDDDDD.”\n\nLabel: 0'

In [130]:
for index, row in df.iterrows():
    text = row['Text']
    completion = fine_tuned.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:personal::AN7TvIij",
        messages=[
            {
                "role": "system", 
                "content": rag_prompt(text)
            },
            {
                "role": "user",
                "content": text
            }
        ]
    )
    tuned.loc[index] = [row['Text'], completion.choices[0].message.content]


In [131]:
tuned

,Text,Toxic
0,Geez forgetful Weve already discussed Marx ana...,0
1,Carioca RFA Thanks support request adminship f...,0
2,Birthday worries Enjoy ur daytalke,0
3,Pseudoscience category Im assuming article pse...,0
4,phrase exists would provided search engine eve...,0
...,...,...
995,2006 UTC rest us ever gone past 3RR good one V...,0
996,Yay lets Pedantic Semantics dance rolls eyes t...,0
997,supposed Know,0
998,guys really discuss napoleon need get fuckin l...,1


In [132]:
compare = tuned['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [133]:
compare

0       True
1       True
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999    False
Name: Toxic, Length: 1000, dtype: bool

In [134]:
# GPT-4o: 97.30% with too mid-detailed evaluation prompt
# GPT-3.5 turbo 50 set: 91.80% with summarized two-shot prompt costing 73 cents
# GPT-3.5 turbo 200 set: 92.00% with summarized two-shot prompt costing 72 cents
# GPT 3.5 turbo 2000 set: 97:30% with summarized two-shot prompt costing 72 cents
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 97.30%


In [135]:

# completion = fine_tuned.chat.completions.create(
#     model="ft:gpt-4o-2024-08-06:personal::AENT6Q3Y",
#     messages=[
#         {
#             "role": "system", 
#             "content": "The user has a toxicity threshold of 1, meaning that they find 90 to 100 percent of comments non-toxic and have extremely high tolerance. Label this text as either 0 for non-toxic or 1 for toxic."
#         },
#         {
#             "role": "user",
#             "content": df['Text'][998]
#         }
#     ]
# )

In [136]:
completion.choices[0].message.content

'1'